In [29]:
import lxml
import datetime
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup 
import requests 

import openpyxl
from openpyxl.styles import NamedStyle,Border, Side,Alignment



ops = Options()

driver = webdriver.Chrome(options=ops)
ops.add_argument('window-size=1920x1080')
ops.add_argument('window-position=500,0')
driver.get("http://youtube.com")

driver.find_element_by_name("search_query").send_keys("indian news latest")
driver.find_element_by_id("search-icon-legacy").click()

wait = WebDriverWait(driver, 10)
for i in range(38):
    wait.until(EC.visibility_of_element_located((By.TAG_NAME,"body"))).send_keys(Keys.END)
    time.sleep(3)
    
WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.ID, "video-title")))
links = driver.find_elements_by_id("video-title")
urls=[]
for x in links:
    urls.append(x.get_attribute("href"))

In [31]:
df = pd.DataFrame(urls)
df_clean=df.dropna(how='any')
df_clean.columns=['urls']
df_clean.reset_index(drop=True, inplace=True)
df_clean.head(500)

,urls
0,https://www.youtube.com/watch?v=KnzQv9Lg6Dw
1,https://www.youtube.com/watch?v=9wUP2i8ehvc
2,https://www.youtube.com/watch?v=3-cRrQRTUag
3,https://www.youtube.com/watch?v=QKYA1l4Srzs
4,https://www.youtube.com/watch?v=3k_JUUCv9rI
...,...
495,https://www.youtube.com/watch?v=BrIAEzsf7OQ
496,https://www.youtube.com/watch?v=8d_89ItBkPY
497,https://www.youtube.com/watch?v=hVG24jxSLVI
498,https://www.youtube.com/watch?v=dajpqzujNF0


In [32]:
headers = {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
def scrape_data(url):
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, "html.parser")
    date = soup.find(class_ = 'watch-time-text').text.replace('Published on ', '').replace('Premiered on ', '')
    views = soup.find("div", attrs={"class": "watch-view-count"}).text[:-6].replace(",", "")
    likes = soup.find("button", attrs={"title": "I like this"}).text.replace(",", "")
    dislikes = soup.find("button", attrs={"title": "I dislike this"}).text.replace(",", "")
    data = {'video_link':url, 'views':views, 'uploaded_date':date, 'likes':likes, 'dislikes':dislikes}
    return data

In [34]:
all_data = []
urls=df_clean['urls'].head(500).values.tolist()
for url in urls:
    all_data.append(scrape_data(url))
df = pd.DataFrame(all_data)
df.to_csv('data1.csv')

In [10]:
df

,video_link,views,uploaded_date,likes,dislikes
0,https://www.youtube.com/watch?v=RwYoYnWVeZo,464,26 Jul 2020,43,13
1,https://www.youtube.com/watch?v=T81ByfKYsGo,819,25 Jul 2020,80,38
2,https://www.youtube.com/watch?v=QKYA1l4Srzs,94681,25 Jul 2020,5511,56
3,https://www.youtube.com/watch?v=3k_JUUCv9rI,,Started streaming on 8 Jul 2020,17645,3305
4,https://www.youtube.com/watch?v=qsuGjzr0aLw,102969,25 Jul 2020,10772,158
5,https://www.youtube.com/watch?v=l9ViEIip9q4,,Started streaming on 2 Mar 2020,898588,159690
6,https://www.youtube.com/watch?v=ckKA-0MUm4E,,Started streaming on 23 Apr 2020,,
7,https://www.youtube.com/watch?v=N0iAioWFu-s,74444,12 Jul 2020,328,64
8,https://www.youtube.com/watch?v=iEu0V3WRiog,68220,22 Jul 2020,1190,79
9,https://www.youtube.com/watch?v=IWSf7jE-sTk,103699,19 Jul 2020,1128,68
